In [1]:
import binascii
from collections import Counter

In [2]:
#from https://joernhees.de/blog/2010/09/21/how-to-convert-hex-strings-to-binary-ascii-strings-in-python-incl-8bit-space/
binary = lambda x: " ".join(reversed( [i+j for i,j in zip( *[ ["{0:04b}".format(int(c,16)) for c in reversed("0"+x)][n::2] for n in [1,0] ] ) ] ))

In [39]:
# Virtual addresses Broadwell evicition set
hex_strings = ['0x00008002','0x0000a002','0x0000c002','0x0000e002','0x00010002']
hex_strings2 = ['0x0000c002','0x00010002','0x00014002','0x00018002','0x0001c002']
bin_strings_v = []
bin_strings_v2 = []

for hex in hex_strings:
    bin_strings_v.append(binary(hex.replace('0x','')))
    
for hex in hex_strings2:
    bin_strings_v2.append(binary(hex.replace('0x','')))
    
print(bin_strings_v)
print(bin_strings_v2)

['00000000 00000000 10000000 00000010', '00000000 00000000 10100000 00000010', '00000000 00000000 11000000 00000010', '00000000 00000000 11100000 00000010', '00000000 00000001 00000000 00000010']
['00000000 00000000 11000000 00000010', '00000000 00000001 00000000 00000010', '00000000 00000001 01000000 00000010', '00000000 00000001 10000000 00000010', '00000000 00000001 11000000 00000010']


In [4]:
def remove_space(strings):
    new_list = []
    for s in strings:
        new_list.append(s.replace(' ',''))
    return new_list

In [40]:
virtual_addresses = remove_space(bin_strings_v)
virtual_addresses2 = remove_space(bin_strings_v2)

In [6]:
def XOR(string, bit1, bit2):
    b1 = string[len(string)-1-bit1]
    b2 = string[len(string)-1-bit2]
    return int(b1)^int(b2)

In [8]:
XOR(virtual_addresses[0], 9, 12)

0

In [9]:
def XOR_HASH(string, start, end):
    length = int((start-end+1)/2)
    xors = []
    for i in range(length):
        xors.append(XOR(string, start-i, start-length-i))
    return xors

In [10]:
XOR_HASH(virtual_addresses[0], 23, 8)

[1, 0, 0, 0, 0, 0, 0, 0]

In [11]:
def XOR7(string):
    return XOR_HASH(string, 26, 12)

def XOR8(string):
    return XOR_HASH(string, 28, 12)

In [12]:
print("XOR-7 on eviction set")
xor7s = []
for address in virtual_addresses:
    xor7s.append(XOR7(address))
    #print(xor7s[len(xor7s)-1])
print(Counter([tuple(i) for i in xor7s]))

print('')
print("XOR-8 on eviction set")
xor8s = []
for address in virtual_addresses:
    xor8s.append(XOR8(address))
    #print(xor8s[len(xor8s)-1])
print(Counter([tuple(i) for i in xor8s]))

XOR-7 on eviction set
Counter({(0, 0, 0, 0, 1, 0, 0): 1, (0, 0, 0, 0, 1, 0, 1): 1, (0, 0, 0, 0, 1, 1, 0): 1, (0, 0, 0, 0, 1, 1, 1): 1, (0, 0, 0, 1, 0, 0, 0): 1})

XOR-8 on eviction set
Counter({(0, 0, 0, 0, 0, 1, 0, 0): 1, (0, 0, 0, 0, 0, 1, 0, 1): 1, (0, 0, 0, 0, 0, 1, 1, 0): 1, (0, 0, 0, 0, 0, 1, 1, 1): 1, (0, 0, 0, 0, 1, 0, 0, 0): 1})


In [37]:
# More uniformely distributed results: XOR-ing the bits that change between our virtual addresses
results = []

for address in virtual_addresses:
    results.append(XOR_HASH(address, 36, 17))
    print(results[len(results)-1])
    
print(Counter([tuple(i) for i in results])) # Check repetitions

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Counter({(0, 0, 0, 1, 0, 0, 0, 0, 0, 0): 5})


In [41]:
results = []

for address in virtual_addresses2:
    results.append(XOR_HASH(address, 36, 17))
    print(results[len(results)-1])
    
print(Counter([tuple(i) for i in results])) # Check repetitions

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Counter({(0, 0, 0, 1, 0, 0, 0, 0, 0, 0): 5})
